In [68]:
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx

import networkx as nx
import numpy as np
import planarity
from tqdm import tqdm

import networkx as nx
import torch

from utils import *

from Dataset import GraphCustomDataset
df_0, df_1, df_2, df_3, df_4, df_5 = load_data()

df_list = [df_0, df_1, df_2, df_3, df_4, df_5]

df_5.drop(columns=['A060000','A002860'],inplace=True)


In [69]:
%%capture 
train_dataset = GraphCustomDataset(df_list, '2017-01-03','2019-02-08', 20, 20, 20, often=True, often_freq = 1)
train_dataset.prepare_data()

KeyboardInterrupt: 

In [ ]:
%%capture 

valid_dataset = GraphCustomDataset(df_list,'2018-02-09', '2020-12-30',20, 20, 20, often=True, often_freq = 1)
valid_dataset.prepare_data()

In [55]:
%%capture 

test_dataset = GraphCustomDataset(df_list,'2022-01-04', '2022-09-30',20, 20, 20, often=True, often_freq = 20) #실제 리밸런싱 할거니깐. 20일에 한번씩 리밸런싱
test_dataset.prepare_data()

학습을 시킬때는 데이터셋이 너무 적으므로, often_freq을 조금 적게 줘서 (즉, 리밸런싱 시점을 다양하게 가져간다) 성능검증 실시. back-testing 에는 이거 뽑는거 자체는 리밸런싱 시점에 대해서만 적용하면 된다.

In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
#from torch_geometric.nn import GATConv
from torch import autograd
import torch.optim as optim
from Dataset import GraphCustomDataset  # Ensure you have this module
from torch.optim.lr_scheduler import StepLR
import copy  # For deep copying the model

from model import GCN_LSTM, EarlyStopping

#early stopping 구현
early_stopping = EarlyStopping(patience=7, verbose=True)

# Hyperparameters (adjust batch_size as needed)
num_node_features = 5
gcn_hidden_dim = 64
lstm_hidden_dim = 128
num_classes = 2
learning_rate = 1e-2
num_epochs = 100
#batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model, optimizer, and loss function setup
model = GCN_LSTM(num_node_features, gcn_hidden_dim, lstm_hidden_dim, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)  # Learning rate scheduler

# Training loop with early stopping and best model checkpointing
best_model_wts = copy.deepcopy(model.state_dict())
best_loss = float('inf')


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for idx in range(len(train_dataset)):
        graphs, labels = train_dataset.get(idx)
        graphs = [graph.to(device) for graph in graphs]
        labels = torch.tensor(labels, dtype=torch.long).to(device)  # Labels as dtype torch.long
        optimizer.zero_grad()
        outputs = model(graphs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for idx in range(len(valid_dataset)):
            graphs, labels = valid_dataset.get(idx)
            graphs = [graph.to(device) for graph in graphs]
            labels = torch.tensor(labels, dtype=torch.long).to(device)
            outputs = model(graphs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(valid_dataset)
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(valid_dataset):.4f}, Val Loss: {val_loss:.4f}, Accuracy: {100*correct/total:.2f}%')
    
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break
    
    scheduler.step()  # Learning rate scheduler step
    
    if val_loss < best_loss:
        best_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())



Epoch 1, Loss: 2.9612, Val Loss: 0.7774, Accuracy: 49.46%
Validation loss decreased, saving model...
Epoch 2, Loss: 2.9827, Val Loss: 0.7046, Accuracy: 49.46%
Validation loss decreased, saving model...
Epoch 3, Loss: 2.9343, Val Loss: 0.7044, Accuracy: 50.54%
Validation loss decreased, saving model...
Epoch 4, Loss: 2.9471, Val Loss: 0.7176, Accuracy: 49.46%
Epoch 5, Loss: 2.9413, Val Loss: 0.7158, Accuracy: 49.46%
Epoch 6, Loss: 2.9407, Val Loss: 0.7158, Accuracy: 49.46%
Epoch 7, Loss: 2.8989, Val Loss: 0.6931, Accuracy: 51.06%
Validation loss decreased, saving model...
Epoch 8, Loss: 2.8996, Val Loss: 0.6931, Accuracy: 51.06%
Validation loss decreased, saving model...
Epoch 9, Loss: 2.8996, Val Loss: 0.6931, Accuracy: 51.06%
Validation loss decreased, saving model...
Epoch 10, Loss: 2.8929, Val Loss: 0.6929, Accuracy: 51.06%
Validation loss decreased, saving model...
Epoch 11, Loss: 2.8918, Val Loss: 0.6929, Accuracy: 51.06%
Epoch 12, Loss: 2.8915, Val Loss: 0.6929, Accuracy: 51.06%


# best model 저장하는 로직, 마코위츠 적용 로직 등 수요일까지 구현. 이후 backtesting 로직 짜기. self.selected_stocks에 있으므로 시점별로 꺼내올 수 있음.

In [57]:
# df_close 상에서의 reabl_idx를 가져올 수 있다.
from model import GCN_LSTM, EarlyStopping
import torch
import torch.nn as nn
# 그떄 그때 꺼내올 수 있도록 리스트로서 저장
num_node_features = 5
gcn_hidden_dim = 64
lstm_hidden_dim = 128
num_classes = 2
learning_rate = 1e-3
num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()


model = GCN_LSTM(num_node_features, gcn_hidden_dim, lstm_hidden_dim, num_classes).to(device)
model.load_state_dict(torch.load('./checkpoints/checkpoint_model.pth'))
model.eval()
test_loss = 0.0
correct = 0
total = 0

#시점별 1로 판단한 stock을 담을 리스트
test_prediction_list = []
test_stock_list = []
with torch.no_grad():
    for idx in range(len(test_dataset)):
        graphs, labels = test_dataset.get(idx)
        graphs = [graph.to(device) for graph in graphs]
        labels = torch.tensor(labels, dtype=torch.long).to(device)
        stocks_list_per_day = test_dataset.stocks_list[idx]
        outputs = model(graphs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        test_prediction_list.append(predicted) # 각 stock들에 대한 판단 결과
        test_stock_list.append(stocks_list_per_day) # 각 시점별 선정된 stock들 티커모음.
        
        
print(f'Test Loss: {test_loss/len(test_dataset):.4f}, Accuracy: {100*correct/total:.2f}%')


Test Loss: 0.6923, Accuracy: 58.33%


In [62]:
stocks_list_per_day

['A005930',
 'A005380',
 'A000660',
 'A015760',
 'A090430',
 'A012330',
 'A032830',
 'A005490',
 'A028260',
 'A055550',
 'A018260',
 'A017670',
 'A035420',
 'A000270',
 'A051910',
 'A105560',
 'A002790',
 'A000810',
 'A051900',
 'A033780',
 'A003550',
 'A034220',
 'A034730',
 'A096770',
 'A009540',
 'A066570',
 'A010130',
 'A068270',
 'A086790',
 'A004020']

In [33]:
len(test_stock_list[0])

30

In [64]:
# make a dictionary of stock and its prediction
up_list = []
down_list = []
for i in range(len(test_stock_list)):
    stock_pred_dict = {} # 시점별로 stock 별 prediction에 대한 dictionary를 생성
    for j in range(len(test_stock_list[i])): # 각 stock 별로 prediction을 dictionary에 저장
        stock_pred_dict[test_stock_list[i][j]] = test_prediction_list[i][j].item()
    # then make a list of stock that has been predicted as 1 and 0 respectively
    stock_pred_1 = []
    stock_pred_0 = []
    for key, value in stock_pred_dict.items():
        if value == 1:
            stock_pred_1.append(key)
        elif value == 0:
            stock_pred_0.append(key)
            
    up_list.append(stock_pred_1)
    down_list.append(stock_pred_0)

In [65]:
outputs

tensor([[-0.0937,  0.0993],
        [ 0.2594, -0.0833],
        [ 0.1619,  0.0685],
        [ 0.1363,  0.1083],
        [ 0.1323,  0.1144],
        [ 0.1318,  0.1153],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154],
        [ 0.1317,  0.1154]], device='cuda:0')

In [67]:
# apply softmax to the output
import torch.nn.functional as F
outputs = F.softmax(outputs, dim=1)
outputs

tensor([[0.4519, 0.5481],
        [0.5848, 0.4152],
        [0.5233, 0.4767],
        [0.5070, 0.4930],
        [0.5045, 0.4955],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959],
        [0.5041, 0.4959]], device='cuda:0')

PORTFOLIO 구축.